In [1]:
import tweepy as tw
import pandas as pd
import pickle
import re
import operator
import itertools 
from configu import credentials
from collections import Counter
import matplotlib.pyplot as plt

In [2]:
api_key = credentials["api_key"]
sec_key = credentials["sec_key"]
access_token = credentials["access_token"]
access_token_secret = credentials["access_token_secret"]

In [3]:
auth = tw.AppAuthHandler(api_key, sec_key)
auth.secure = True
api = tw.API(auth, wait_on_rate_limit=True)

In [4]:
main_ht = []
for t in tw.Cursor(api.search, q="#Covid19IndiaHelp", lang="en", tweet_mode='extended').items(2000):
    main_ht.append(t)

In [5]:
open_file = open("main_hashtag.txt", "wb")
pickle.dump(main_ht, open_file)
open_file.close()

open_file = open("main_hashtag.txt", "rb")
main_ht = pickle.load(open_file)
open_file.close()

In [ ]:
other_ht = []
top10_ht = []

for tweet in main_ht:
    tw_ht = []
    for ht in tweet.entities['hashtags']:
        if (ht['text'] != "Covid19IndiaHelp"):
            tw_ht.append(ht['text'])
    other_ht.extend(list(set(tw_ht)))

occurence_count = Counter(other_ht)
for ht in occurence_count.most_common(10):
    top10_ht.append(ht[0])
    
print(top10_ht)

In [7]:
def get_tweets(ht):
    tweets_for_ht = []
    for t in tw.Cursor(api.search, q=ht, lang="en", tweet_mode='extended').items(2000):
        tweets_for_ht.append(t)
    return tweets_for_ht

dict for top 10 hashtags containing their tweets (2000 each)
ht_dict = {}
for ht in top10_ht:
    ht_dict[ht] = get_tweets(ht)

In [8]:
ht_dict['Covid19IndiaHelp'] = main_ht
open_file = open("sec2_all_tweets_dict.txt", "wb")
pickle.dump(ht_dict, open_file)
open_file.close()

open_file = open("sec2_all_tweets_dict.txt", "rb")
ht_dict = pickle.load(open_file)
open_file.close()

In [9]:
all_tweets = []

for tw_list in ht_dict.values():
    all_tweets.extend(tw_list)

most_occ = {}
most_likes = {}
most_rt = {}

for tweet in all_tweets:
    temp_ht = []
    
    for ht in tweet.entities['hashtags']:
        if (ht['text'] != "Covid19IndiaHelp"):
            temp_ht.append(ht['text'])
            
    for uniq_ht in (list(set(temp_ht))):
        
        #most number of occurrences
        if(uniq_ht in most_occ):
            most_occ[uniq_ht] += 1
        else:
            most_occ[uniq_ht] = 1
    
        #most number of likes
        if(uniq_ht in most_likes):
            most_likes[uniq_ht] += tweet.favorite_count
        else:
            most_likes[uniq_ht] = tweet.favorite_count
            
        #most number of retweets
        if(uniq_ht in most_rt):
            most_rt[uniq_ht] += tweet.retweet_count
        else:
            most_rt[uniq_ht] = tweet.retweet_count
    

most_occ = dict(sorted(most_occ.items(), key=operator.itemgetter(1),reverse=True))
topmost_occ = dict(itertools.islice(most_occ.items(),10))

most_likes = dict(sorted(most_likes.items(), key=operator.itemgetter(1),reverse=True))
topmost_likes = dict(itertools.islice(most_likes.items(),10))

most_rt = dict(sorted(most_rt.items(), key=operator.itemgetter(1),reverse=True))
topmost_rt = dict(itertools.islice(most_rt.items(),10))

In [10]:
plt.figure(figsize=(14,5))
barlist1 = plt.bar(topmost_occ.keys(), topmost_occ.values())
plt.xlabel('Hashtag')
plt.ylabel('Number of occurrences')
plt.show()

In [11]:
plt.figure(figsize=(16,6))
barlist2 = plt.bar(topmost_likes.keys(), topmost_likes.values())
plt.xlabel('Hashtag')
plt.ylabel('Number of likes')
plt.show()

In [12]:
plt.figure(figsize=(16,6))
barlist2 = plt.bar(topmost_rt.keys(), topmost_rt.values())
plt.xlabel('Hashtag')
plt.ylabel('Number of retweets')
plt.show()

In [ ]:
def R_calc(hashtag):
    no_of_t = 0
    no_of_rt = 0
    for tweet in all_tweets:
        have_hash = False
        for ht in tweet.entities['hashtags']:
            if(hashtag in ht.values()):
                have_hash = True
                no_of_t += 1
        try:
            if(tweet.retweeted_status and have_hash):
                no_of_rt += 1
        except:
            continue        
    return (no_of_rt*100)/no_of_t

def F_calc(hashtag):
    
    total_users = {}
    tu_tweets = 0
    for tweet in all_tweets:
        for ht in tweet.entities['hashtags']:
            if(hashtag in ht.values()):
                if(tweet.user.id in total_users):
                    total_users[tweet.user.id] += 1
                else:
                    total_users[tweet.user.id] = 1
    
    total_users = dict(sorted(total_users.items(), key=operator.itemgetter(1),reverse=True))
    
    traffic_50 = 0
    traffic_total = 0
    count = 0
    for i in total_users.values():
        count += 1
        traffic_total += i
        if(count < 50):
            traffic_50 += i
    return (traffic_50*100)/traffic_total

def U_calc(hashtag):
    total_users = {}
    total_tweets = 0
    for tweet in all_tweets:
        for ht in tweet.entities['hashtags']:
            if(hashtag in ht.values()):
                if(tweet.user.id in total_users):
                    total_users[tweet.user.id] += 1
                else:
                    total_users[tweet.user.id] = 1
                    
    for i in total_users.values():
        total_tweets += i
    return total_tweets/len(total_users)
    
R_dict_occ = {}
F_dict_occ = {}
U_dict_occ = {}

R_dict_likes = {}
F_dict_likes = {}
U_dict_likes = {}

R_dict_rt = {}
F_dict_rt = {}
U_dict_rt = {}

for hashtag in topmost_occ.keys():
    R_dict_occ[hashtag] = R_calc(hashtag)
    F_dict_occ[hashtag] = F_calc(hashtag)
    U_dict_occ[hashtag] = U_calc(hashtag)
    
for hashtag in topmost_likes.keys():
    R_dict_likes[hashtag] = R_calc(hashtag)
    F_dict_likes[hashtag] = F_calc(hashtag)
    U_dict_likes[hashtag] = U_calc(hashtag)
    
for hashtag in topmost_rt.keys():
    R_dict_rt[hashtag] = R_calc(hashtag)
    F_dict_rt[hashtag] = F_calc(hashtag)
    U_dict_rt[hashtag] = U_calc(hashtag)

In [ ]:
ctm_occ = {}
ctm_likes = {}
ctm_rt = {}

for i in range(10):
    top_occ = list(topmost_occ.keys())
    ht_occ = top_occ[i]
    ctm_occ[ht_occ] = (R_dict_occ[ht_occ]/10) + F_dict_occ[ht_occ] + U_dict_occ[ht_occ]
    
    top_likes = list(topmost_likes.keys())
    ht_likes = top_likes[i]
    ctm_likes[ht_likes] = (R_dict_likes[ht_likes]/10) + F_dict_likes[ht_likes] + U_dict_likes[ht_likes]
    
    top_rt = list(topmost_rt.keys())
    ht_rt = top_rt[i]
    ctm_rt[ht_rt] = (R_dict_rt[ht_rt]/10) + F_dict_rt[ht_rt] + U_dict_rt[ht_rt]

print(ctm_occ)
print(ctm_likes)
print(ctm_rt)